In [ ]:
%env DATA_BUCKET=tbd-2024l-9910-data
%env GEN_OUTPUT_DIR=/tmp/tpc-di
%env REPO_ROOT=/home/jupyter/git/tbd-tpc-di/

In [ ]:
!pip3.8 install typer[All]==0.9.0 google-cloud-storage==2.13.0

## Install SDKMAN for setting up JVM 8 enviroment

In [ ]:
%%bash
curl -s https://get.sdkman.io | bash

## Install and set as default JVM 8

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 8.0.392-amzn
sdk use java 8.0.392-amzn

## Check if JVM 8 is used

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
java -version

## Clone tbd-tpc-di repo

In [ ]:
%%bash
mkdir -p git && cd git
git clone https://github.com/mwiewior/tbd-tpc-di.git
cd tbd-tpc-di
git pull

## Generate input dataset (run this cell below from the terminal!!!)
It should take approx. 15min with scale factor set to 100 and generate approx. 10GiB

In [ ]:
%%script bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd /home/jupyter/git/tbd-tpc-di/tools/ 
java -jar DIGen.jar -sf 100 -o /tmp/tpc-di

In [ ]:
# Install and setup JVM 11

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 11.0.21-amzn
sdk use java 11.0.21-amzn -y

## Load staging

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd $REPO_ROOT
python3.8 tpcdi.py --output-directory $GEN_OUTPUT_DIR --stage $DATA_BUCKET

## Run dbt ELT

In [ ]:
%%bash
cd $REPO_ROOT
dbt deps
dbt run

## Run dbt tests

In [ ]:
%%bash
cd $REPO_ROOT
dbt test

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TBD-TPC-DI-setup") \
    .master("yarn") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
spark.sql("show databases").show()

In [ ]:
spark.sql("use demp_gold")

In [ ]:
spark.sql("show tables").show()

In [ ]:
spark.stop()